In [2]:
import tweepy
import re
import csv
import glob
from itertools import islice
import os
import os, sys
import wget
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter


screen_names=[]
parties=[]

lidmaatschap = {}

with open('/home/linda/manifestos/tweets/NL/NL_twitter_mp.csv', encoding='utf-8',mode='r',newline='') as twitterlist:
    reader = csv.reader(twitterlist,delimiter=',')
    for row in csv.reader(islice(twitterlist, None)):
        screen_names.append(row[0])
        parties.append(row[1])
        lidmaatschap[row[0]] = row[1]

In [3]:
auth = tweepy.OAuthHandler('RuYN0e3wWRrLpnvA8X2QFyLtH', 'qxs2ZdcPbnwB0kQlbgiygzF1LI3B3rcYA7pNaw1ApaMDj3xFDR',)
auth.set_access_token('1398102110-z9BBCXR9QAeNdXjnPTnrUwKRodGx8OVJzGiCGGi', 'RMVd2CFC3O8NjTDbgG3FVdj9BTV5NqWSkFuLk1Kl9ymd2')
api = tweepy.API(auth)

In [ ]:
# Note: script hieronder werkt bijna helemaal:
# images worden gedownload, opgeslagen en geanalyseerd
# text uit images wordt opgeslagen en gekoppeld aan tweet
# krijg alleen nu niet voor elkaar om die text samen te voegen met de text van de tweet zelf
# zou zijn omdat een van beide een list is, maar ik weet eigenlijk niet welke.

In [6]:
for screen_name in screen_names[:1]:
    print("Starting to get tweets from {}".format(screen_name))
    os.makedirs("/home/linda/manifestos/tweets/downloads/NL/Party leaders/test/images_{}_{}".format(screen_name,lidmaatschap[screen_name]))    
    os.chdir("/home/linda/manifestos/tweets/downloads/NL/Party leaders/test/images_{}_{}".format(screen_name,lidmaatschap[screen_name]))    
    alltweets = []	
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    alltweets.extend(new_tweets)
    oldest = alltweets[-1].id - 1
    while len(new_tweets) > 0:
        print ("getting tweets before %s" % (oldest))
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        alltweets.extend(new_tweets)	
        oldest = alltweets[-1].id - 1
        print ("...%s tweets downloaded so far" % (len(alltweets)))
    cleaned_text = [re.sub(r'http[s]?:\/\/.*[\W]*', '', i.text, flags=re.MULTILINE) for i in alltweets] # remove urls
    cleaned_text = [re.sub(r'@[\w]*', '', i, flags=re.MULTILINE) for i in cleaned_text] # remove the @twitter mentions 
    cleaned_text = [re.sub(r'RT.*','', i, flags=re.MULTILINE) for i in cleaned_text] # delete the retweets
    picturetext=[]
    totaltext = []
    pictures=[]
    for tweet in alltweets:
        if 'media' in tweet.entities:
            for image in  tweet.entities['media']:
                picture = image['media_url']
                wget.download(picture)
                pic = picture.split("/")[-1] 
                im = Image.open(pic)
                im = im.filter(ImageFilter.MedianFilter())
                enhancer = ImageEnhance.Contrast(im)
                im = enhancer.enhance(2)
                im = im.convert('1')
                im.save('temp.jpg')
                text_from_picture = pytesseract.image_to_string(Image.open('temp.jpg'))
                os.remove('temp.jpg')
        elif 'media' not in tweet.entities:
            picture = 'no picture'
            text_from_picture = ''
        picturetext.append(text_from_picture)
        for i in cleaned_text:
            ttext = text_from_picture + i
        totaltext.append(ttext)
        pictures.append(picture)
    outtweets = [[tweet.id_str, tweet.favorite_count, tweet.retweet_count, tweet.created_at, pictures[idx], picturetext[idx],totaltext[idx],cleaned_text[idx]] for idx,tweet in enumerate(alltweets)]
    outputfile = '/home/linda/manifestos/tweets/downloads/NL/Party leaders/test/{}_{}_tweets_images.csv'.format(screen_name,lidmaatschap[screen_name])
    print("Writing stuff to {}".format(outputfile))
    with open(outputfile, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","favourite_count","retweet_count", "created_at", "image", "imagetext", "totaltext", "cleaned_text"])
        writer.writerows(outtweets)

pass



Starting to get tweets from geertwilderspvv


FileExistsError: [Errno 17] File exists: '/home/linda/manifestos/tweets/downloads/NL/Party leaders/test/images_geertwilderspvv_PVV'